<a href="https://colab.research.google.com/github/Linked1-List/AEandLG/blob/main/AEandLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import librosa
import os
import soundfile as sf

# 음성 파일을 로드하는 함수
def load_audio(file_path, target_sr=16000):
    audio, sr = librosa.load(file_path, sr=target_sr)
    return audio, sr

# 음성을 청크로 나누는 함수
def split_audio(audio, sr, chunk_length_sec=5, output_dir='output_audio'):
    # chunk 길이를 샘플로 변환 (초 -> 샘플)
    chunk_length_samples = chunk_length_sec * sr

    # 저장할 디렉토리 생성 (없으면)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 청크로 나누어서 저장
    chunks = []
    for i in range(0, len(audio), chunk_length_samples):
        chunk = audio[i:i+chunk_length_samples]
        chunk_filename = f'chunk_{i // chunk_length_samples + 1}.wav'
        chunk_path = os.path.join(output_dir, chunk_filename)
        sf.write(chunk_path, chunk, sr)  # chunk 저장
        chunks.append(chunk_path)

    return chunks

# 메인 실행 부분
def main(input_audio_path, output_dir='output_audio', chunk_length_sec=5):
    # 오디오 파일 로드
    audio, sr = load_audio(input_audio_path)

    # 오디오를 청크로 나누고 저장
    chunks = split_audio(audio, sr, chunk_length_sec, output_dir)

    # 저장된 청크 파일 리스트 출력
    print(f'총 {len(chunks)}개의 청크가 {output_dir} 디렉토리에 저장되었습니다.')
    return chunks

# 사용 예시
input_audio_path = '/content/drive/MyDrive/data/chosubin.mp3'  # 원본 음성 파일 경로
output_dir = '/content/drive/MyDrive/humanchunk'     # 분할된 파일을 저장할 디렉토리
chunks = main(input_audio_path, output_dir, chunk_length_sec=5)


총 203개의 청크가 /content/drive/MyDrive/humanchunk 디렉토리에 저장되었습니다.


In [ ]:
import librosa
import os
import soundfile as sf

# 음성 파일을 로드하는 함수
def load_audio(file_path, target_sr=16000):
    audio, sr = librosa.load(file_path, sr=target_sr)
    return audio, sr

# 음성을 청크로 나누는 함수
def split_audio(audio, sr, chunk_length_sec=5, output_dir='output_audio'):
    # chunk 길이를 샘플로 변환 (초 -> 샘플)
    chunk_length_samples = chunk_length_sec * sr

    # 저장할 디렉토리 생성 (없으면)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 청크로 나누어서 저장
    chunks = []
    for i in range(0, len(audio), chunk_length_samples):
        chunk = audio[i:i+chunk_length_samples]
        chunk_filename = f'chunk_{i // chunk_length_samples + 1}.wav'
        chunk_path = os.path.join(output_dir, chunk_filename)
        sf.write(chunk_path, chunk, sr)  # chunk 저장
        chunks.append(chunk_path)

    return chunks

# 메인 실행 부분
def main(input_audio_path, output_dir='output_audio', chunk_length_sec=5):
    # 오디오 파일 로드
    audio, sr = load_audio(input_audio_path)

    # 오디오를 청크로 나누고 저장
    chunks = split_audio(audio, sr, chunk_length_sec, output_dir)

    # 저장된 청크 파일 리스트 출력
    print(f'총 {len(chunks)}개의 청크가 {output_dir} 디렉토리에 저장되었습니다.')
    return chunks

# 사용 예시
input_audio_path = '/content/drive/MyDrive/data/새+프로젝트.mp3'  # 원본 음성 파일 경로
output_dir = '/content/drive/MyDrive/aichunk'     # 분할된 파일을 저장할 디렉토리
chunks = main(input_audio_path, output_dir, chunk_length_sec=5)


총 202개의 청크가 /content/drive/MyDrive/aichunk 디렉토리에 저장되었습니다.


In [ ]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchaudio.transforms as T

# 오토인코더 모델 정의
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 음성 파일을 Mel-spectrogram으로 변환하는 함수
def audio_to_mel_spectrogram(waveform, sample_rate=16000, target_length=16000):
    # Mel-spectrogram 변환
    mel_spec = T.MelSpectrogram(sample_rate=sample_rate)(waveform)  # Mel-Spectrogram 생성
    mel_spec_db = T.AmplitudeToDB()(mel_spec)  # 로그 스펙트로그램으로 변환

    # 패딩 또는 자르기 (길이를 target_length로 맞추기)
    num_samples = mel_spec_db.shape[-1]

    if num_samples < target_length:
        # 패딩: 짧은 음성 데이터를 0으로 채워서 길이를 맞춤
        padding = target_length - num_samples
        mel_spec_db = torch.cat([mel_spec_db, torch.zeros(1, mel_spec_db.shape[1], padding)], dim=-1)
    elif num_samples > target_length:
        # 자르기: 긴 음성 데이터를 target_length로 잘라냄
        mel_spec_db = mel_spec_db[:, :, :target_length]

    return mel_spec_db

# 음성 파일 로드 및 전처리 함수
def load_audio_file(file_path, sample_rate=16000, target_length=16000):
    waveform, original_sample_rate = torchaudio.load(file_path)

    # 샘플링 레이트가 다르면 변환
    if original_sample_rate != sample_rate:
        waveform = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=sample_rate)(waveform)

    # 정규화: 음성 데이터를 [-1, 1] 범위로 정규화
    waveform = waveform / waveform.abs().max()

    # Mel-spectrogram으로 변환 및 길이 맞추기
    mel_spectrogram = audio_to_mel_spectrogram(waveform, sample_rate, target_length)

    return mel_spectrogram

# 데이터셋 로드 함수
def load_dataset(file_paths, sample_rate=16000, target_length=16000):
    spectrograms = []
    for file_path in file_paths:
        mel_spectrogram = load_audio_file(file_path, sample_rate, target_length)
        spectrograms.append(mel_spectrogram)
    return torch.stack(spectrograms)  # [N, 1, H, W] 형식으로 반환

# 데이터 로딩
human_data_dir = '/content/drive/MyDrive/humanchunk'
human_files = [os.path.join(human_data_dir, f) for f in os.listdir(human_data_dir) if f.endswith('.wav')]
human_spectrograms = load_dataset(human_files)

ai_data_dir = '/content/drive/MyDrive/aichunk'
ai_files = [os.path.join(ai_data_dir, f) for f in os.listdir(ai_data_dir) if f.endswith('.wav')]
ai_spectrograms = load_dataset(ai_files)

# DataLoader 설정
batch_size = 8
human_dataset = TensorDataset(human_spectrograms)
dataloader = DataLoader(human_dataset, batch_size=batch_size, shuffle=True)

# 모델 및 손실 함수, 옵티마이저 설정
model = ConvAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
epochs = 20
for epoch in range(epochs):
    for batch in dataloader:
        input_spectrograms = batch[0]

        # 모델 예측 및 손실 계산
        decoded = model(input_spectrograms)
        loss = criterion(decoded, input_spectrograms)

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {loss.item():.4f}')

# 재구성 오차 계산 함수
def calculate_reconstruction_error(spectrogram):
    with torch.no_grad():
        decoded = model(spectrogram.unsqueeze(0))  # [1, 1, H, W] 형식으로 입력
        error = criterion(decoded, spectrogram.unsqueeze(0)).item()
    return error

# 인간 음성의 개별 재구성 오차 출력
print("\nHuman Voice Reconstruction Errors:")
humansum=0
cnth=0
for i, human_spectrogram in enumerate(human_spectrograms):
    error = calculate_reconstruction_error(human_spectrogram)
    print(f"Human Sample {i+1}: Reconstruction Error = {error:.4f}")
    if error>=0:
      humansum+=error
      cnth=i+1
print(humansum/cnth)

# AI 음성의 개별 재구성 오차 출력
print("\nAI Voice Reconstruction Errors:")
aisum=0
cnta=0
for i, ai_spectrogram in enumerate(ai_spectrograms):
    error = calculate_reconstruction_error(ai_spectrogram)
    print(f"AI Sample {i+1}: Reconstruction Error = {error:.4f}")
    if error>=0:
      aisum+=error
      cnta=i+1
print(aisum/cnta)


/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Epoch [1/20], Training Loss: 21.4885
Epoch [2/20], Training Loss: 21.6023
Epoch [3/20], Training Loss: 22.6085
Epoch [4/20], Training Loss: 20.8773
Epoch [5/20], Training Loss: 23.6110
Epoch [6/20], Training Loss: 21.0165
Epoch [7/20], Training Loss: 20.3157
Epoch [8/20], Training Loss: 19.5593
Epoch [9/20], Training Loss: 20.8824
Epoch [10/20], Training Loss: 21.2178
Epoch [11/20], Training Loss: 21.6284
Epoch [12/20], Training Loss: 21.2100
Epoch [13/20], Training Loss: 23.2248
Epoch [14/20], Training Loss: 19.8919
Epoch [15/20], Training Loss: 19.5243
Epoch [16/20], Training Loss: 22.8506
Epoch [17/20], Training Loss: 21.6857
Epoch [18/20], Training Loss: 20.2907
Epoch [19/20], Training Loss: 22.5964
Epoch [20/20], Training Loss: 22.7092

Human Voice Reconstruction Errors:
Human Sample 1: Reconstruction Error = 19.0515
Human Sample 2: Reconstruction Error = 16.5600
Human Sample 3: Reconstruction Error = 17.5154
Human Sample 4: Reconstruction Error = 17.1839
Human Sample 5: Reconstru

In [ ]:
# 모델 학습 완료 후
model_save_path = "/content/drive/MyDrive/autoencoder/autoencoder_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


NameError: name 'model' is not defined

In [ ]:
import librosa
import os
import soundfile as sf

# 음성 파일을 로드하는 함수
def load_audio(file_path, target_sr=16000):
    audio, sr = librosa.load(file_path, sr=target_sr)
    return audio, sr

# 음성을 청크로 나누는 함수
def split_audio(audio, sr, chunk_length_sec=5, output_dir='output_audio'):
    # chunk 길이를 샘플로 변환 (초 -> 샘플)
    chunk_length_samples = chunk_length_sec * sr

    # 저장할 디렉토리 생성 (없으면)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 청크로 나누어서 저장
    chunks = []
    for i in range(0, len(audio), chunk_length_samples):
        chunk = audio[i:i+chunk_length_samples]
        chunk_filename = f'chunk_{i // chunk_length_samples + 1}.wav'
        chunk_path = os.path.join(output_dir, chunk_filename)
        sf.write(chunk_path, chunk, sr)  # chunk 저장
        chunks.append(chunk_path)

    return chunks

# 메인 실행 부분
def main(input_audio_path, output_dir='output_audio', chunk_length_sec=5):
    # 오디오 파일 로드
    audio, sr = load_audio(input_audio_path)

    # 오디오를 청크로 나누고 저장
    chunks = split_audio(audio, sr, chunk_length_sec, output_dir)

    # 저장된 청크 파일 리스트 출력
    print(f'총 {len(chunks)}개의 청크가 {output_dir} 디렉토리에 저장되었습니다.')
    return chunks

# 사용 예시
input_audio_path = '/content/drive/MyDrive/data/새+프로젝트 (2).mp3'  # 원본 음성 파일 경로
output_dir = '/content/drive/MyDrive/data/ai'     # 분할된 파일을 저장할 디렉토리
chunks = main(input_audio_path, output_dir, chunk_length_sec=2)


총 129개의 청크가 /content/drive/MyDrive/data/ai 디렉토리에 저장되었습니다.


In [ ]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchaudio.transforms as T

# 오토인코더 모델 정의
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 음성 파일을 Mel-spectrogram으로 변환하는 함수
def audio_to_mel_spectrogram(waveform, sample_rate=16000, target_length=16000):
    # Mel-spectrogram 변환
    mel_spec = T.MelSpectrogram(sample_rate=sample_rate)(waveform)  # Mel-Spectrogram 생성
    mel_spec_db = T.AmplitudeToDB()(mel_spec)  # 로그 스펙트로그램으로 변환

    # 패딩 또는 자르기 (길이를 target_length로 맞추기)
    num_samples = mel_spec_db.shape[-1]

    if num_samples < target_length:
        # 패딩: 짧은 음성 데이터를 0으로 채워서 길이를 맞춤
        padding = target_length - num_samples
        mel_spec_db = torch.cat([mel_spec_db, torch.zeros(1, mel_spec_db.shape[1], padding)], dim=-1)
    elif num_samples > target_length:
        # 자르기: 긴 음성 데이터를 target_length로 잘라냄
        mel_spec_db = mel_spec_db[:, :, :target_length]

    return mel_spec_db

# 음성 파일 로드 및 전처리 함수
def load_audio_file(file_path, sample_rate=16000, target_length=16000):
    waveform, original_sample_rate = torchaudio.load(file_path)

    # 샘플링 레이트가 다르면 변환
    if original_sample_rate != sample_rate:
        waveform = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=sample_rate)(waveform)

    # 정규화: 음성 데이터를 [-1, 1] 범위로 정규화
    waveform = waveform / waveform.abs().max()

    # Mel-spectrogram으로 변환 및 길이 맞추기
    mel_spectrogram = audio_to_mel_spectrogram(waveform, sample_rate, target_length)

    return mel_spectrogram

# 데이터셋 로드 함수
def load_dataset(file_paths, sample_rate=16000, target_length=16000):
    spectrograms = []
    for file_path in file_paths:
        mel_spectrogram = load_audio_file(file_path, sample_rate, target_length)
        spectrograms.append(mel_spectrogram)
    return torch.stack(spectrograms)  # [N, 1, H, W] 형식으로 반환

criterion = nn.MSELoss()

# 재구성 오차 계산 함수
def calculate_reconstruction_error(spectrogram):
    with torch.no_grad():
        decoded = model(spectrogram.unsqueeze(0))  # [1, 1, H, W] 형식으로 입력
        error = criterion(decoded, spectrogram.unsqueeze(0)).item()
    return error

# 모델 로드
model_save_path = "/content/drive/MyDrive/autoencoder/autoencoder_model.pth"
model = ConvAutoencoder()
model.load_state_dict(torch.load(model_save_path))
model.eval()

# 데이터 로딩
new_data_dir = '/content/drive/MyDrive/data/ai'
new_files = [os.path.join(new_data_dir, f) for f in os.listdir(new_data_dir) if f.endswith('.wav')]
new_spectrograms = load_dataset(new_files)

# 새 음성의 개별 재구성 오차 출력
print("\nNew Voice Reconstruction Errors:")
nsum=0
cntn=0
for i, new_spectrogram in enumerate(new_spectrograms):
    error = calculate_reconstruction_error(new_spectrogram)
    print(f"New Sample {i+1}: Reconstruction Error = {error:.4f}")
    if error>=0:
      nsum+=error
      cntn=i+1
print(nsum/cntn)

/usr/local/lib/python3.11/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(



New Voice Reconstruction Errors:
New Sample 1: Reconstruction Error = 49.7207
New Sample 2: Reconstruction Error = 20.9204
New Sample 3: Reconstruction Error = 20.0723
New Sample 4: Reconstruction Error = 44.3286
New Sample 5: Reconstruction Error = 11.9431
New Sample 6: Reconstruction Error = 19.0803
New Sample 7: Reconstruction Error = 19.4096
New Sample 8: Reconstruction Error = 23.9484
New Sample 9: Reconstruction Error = 43.2083
New Sample 10: Reconstruction Error = 16.8326
New Sample 11: Reconstruction Error = 17.1320
New Sample 12: Reconstruction Error = 39.1951
New Sample 13: Reconstruction Error = 34.1409
New Sample 14: Reconstruction Error = 31.7406
New Sample 15: Reconstruction Error = 35.3160
New Sample 16: Reconstruction Error = 28.5766
New Sample 17: Reconstruction Error = 22.6037
New Sample 18: Reconstruction Error = 16.3649
New Sample 19: Reconstruction Error = 52.9520
New Sample 20: Reconstruction Error = 36.7415
New Sample 21: Reconstruction Error = 62.0864
New Sampl

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
)
from sklearn.preprocessing import StandardScaler

# 1. CSV 파일 불러오기
df = pd.read_csv("/content/drive/MyDrive/data/reconstruction_errors.csv")  # 같은 디렉토리에 있는 파일 사용

# 2. 데이터 전처리
errors = df["error"].values
labels = df["label"].values

# 3. 스케일링
scaler = StandardScaler()
errors_scaled = scaler.fit_transform(errors.reshape(-1, 1))

# 4. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(errors_scaled, labels, test_size=0.2, random_state=42)

# 5. 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 6. 예측 및 평가
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
cm = confusion_matrix(y_test, y_pred)

# 7. 평가 결과 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# 8. 특정 Reconstruction Error에 대한 예측 확률 함수
def predict_probabilities(reconstruction_error):
    scaled_error = scaler.transform(np.array([[reconstruction_error]]))
    probabilities = model.predict_proba(scaled_error)
    human_prob, ai_prob = probabilities[0]
    return human_prob, ai_prob

# 9. 예시 사용
test_error = 53.9091
human_prob, ai_prob = predict_probabilities(test_error)

print(f"\nReconstruction Error = {test_error}")
print(f"Probability of Human: {human_prob:.2%}")
print(f"Probability of AI: {ai_prob:.2%}")


Accuracy: 0.8689
Precision: 0.8889
Recall (Sensitivity): 0.7273
F1 Score: 0.8000

Reconstruction Error = 53.9091
Probability of Human: 17.10%
Probability of AI: 82.90%
